In [1]:
import pandas as pd

In [ ]:
books = pd.read_csv("../data/cleaned-books.csv")

In [8]:
books["categories"].value_counts().reset_index().query("count>50")

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Philosophy,117
6,Religion,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [9]:
catergory_maping = {
    "Fiction":"Fiction",
	"Juvenile Fiction": "Childern Fiction",
	"Biography & Autobiography": "Nonfiction",
	"History":  "Nonfiction",
	"Literary Criticism":  "Nonfiction",
	"Philosophy":  "Nonfiction",
	"Religion":  "Nonfiction",
	"Comics & Graphic Novels":  "Fiction",
	"Drama": "Fiction",
	"Juvenile Nonfiction": "Childern Nonfiction",
	"Science":  "Nonfiction",
	"Poetry": "Fiction"
}

books["simple_categories"] = books["categories"].map(catergory_maping)

In [13]:
from transformers import pipeline

fiction_categories = ["Fiction", "Nonfiction"]

pipe = pipeline("zero-shot-classification",
                model= "facebook/bart-large-mnli")

Device set to use cpu


In [16]:
sequence = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[0]

In [17]:
pipe(sequence, fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [19]:
import numpy as np

max_index = np.argmax(pipe(sequence, fiction_categories)["scores"])
max_label = pipe(sequence, fiction_categories)["labels"][max_index]
max_label

'Fiction'

In [20]:
def generate_prediction(sequence, categories):
    prediction = pipe(sequence, categories)
    max_index = np.argmax(prediction["scores"])
    max_label = prediction["labels"][max_index]
    return max_label

In [22]:
from tqdm import tqdm

actual_cats = []
predicted_cats = []

for i in tqdm(range(0, 300)):
    sequence = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop = True)[i]
    predicted_cats += [generate_prediction(sequence, fiction_categories)]
    actual_cats += ["Fiction"]

100%|██████████| 300/300 [07:50<00:00,  1.57s/it]


In [23]:
for i in tqdm(range(0, 300)):
    sequence = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop = True)[i]
    predicted_cats += [generate_prediction(sequence, fiction_categories)]
    actual_cats += ["Nonfiction"]

100%|██████████| 300/300 [06:49<00:00,  1.36s/it]


In [24]:
prediction_df = pd.DataFrame({"actual_categories": actual_cats, "predicted_categories": predicted_cats})
prediction_df.head()

,actual_categories,predicted_categories
0,Fiction,Fiction
1,Fiction,Fiction
2,Fiction,Fiction
3,Fiction,Nonfiction
4,Fiction,Fiction


In [25]:
prediction_df["correct_prediction"] = (
    np.where(prediction_df["actual_categories"] == prediction_df["predicted_categories"], True, False)
)

In [26]:
prediction_df["correct_prediction"].sum() / len(prediction_df)

0.5

In [27]:
isbns = []
predicted_cats = []

missing_cats = books.loc[books["simple_categories"].isna(), ["isbn13", "description"]].reset_index(drop = True)

In [28]:
for i in tqdm(range(0, len(missing_cats))):
    sequence = missing_cats["description"][i]
    predicted_cats += [generate_prediction(sequence, fiction_categories)]
    isbns += [missing_cats["isbn13"][i]]

100%|██████████| 1454/1454 [27:07<00:00,  1.12s/it] 


In [29]:
missing_predicted_df = pd.DataFrame({"isbn13":isbns, "predicted_categories": predicted_cats})

In [30]:
books = pd.merge(books, missing_predicted_df, on = "isbn13", how = "left")

books["simple_categories"] = np.where(books["simple_categories"].isna(), books["predicted_categories"], books["simple_categories"])
books = books.drop("predicted_categories", axis = 1)

In [31]:
books["simple_categories"].isna().sum()

0

In [32]:
books.to_csv("../data/books_with_categories.csv", index = False)